In [ ]:
!pip install pymorphy2
!pip install pymystem3
!pip install natasha
!pip install spacy
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz


В качестве текстов на русском языке для POS-тэггинга я взяла посты из паблика Дубков, потому что в нем очень много студенческого арго, аббревиатур, разговорных слов, локальных новостей, шуток. Большое количество слов из постов этого паблика являются несловарными и едва ли встречаются за пределами дубковского сообщества: "молодежка" - автобус от-до метро Молодежная, "славянка" - автобус от и до метро Славянский бульвар, "Трилист" - не-дубковское общежитие, расположенное возле ж/д станции Одинцово. Само Одинцово периодически зовется "Оди". И так далее.

Тематика постов разнообразна и охватывает как учебные вопросы, так и бытовые, например, обмен вещами или расписание автобусов.

Кроме того, далеко не во всех постах паблика соблюдаются правила пунктуации, что может усложнить работу POS-тэггеров.

In [4]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np
import string
import spacy
from collections import defaultdict as dd
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# тут я открываю файлик с миникорпусом, я его собрала вручную, поискав самые любопытные отрывки постов последних месяцев
with open('Корпус.txt', encoding = 'utf-8') as file:
  text = file.read()
text_tokenized = word_tokenize(text)

In [ ]:
# здесь я размечаю вручную все токены, не советую запускать ячейку, а то тоже придется зависнуть с разметкой
gold = []
for word in text_tokenized:
  print(word)
  gold.append(input())

Комментарий про тэгсет! Я решила взять тэги с https://universaldependencies.org/treebanks/ru_syntagrus/index.html, потому что там хорошие примеры, позволяющие сверить часть спорных случаев, во многих планах универсальная разметка, которую будет легко адаптировать под разные POS-тэггеры, а еще у них удобные нечастеречные тэги PUNCT, NUM и X, которые очень уместно было использовать. Но пометка: у меня к NUM относятся штуки типа обозначения времени в формате '00:00' и числительные, что логично, а Х у меня всего один, это "3к (aka третий корпус), и сделала я так, потому что часть речи у этого словосочетания не очень-то и определишь. Кроме того, я не использовала тег PROPN, так как для многих вещей в Дубках не очень ясно, это имя собственное или нарицательное, например, "славянку" никогда не пишут с большой буквы и фактически именем собственным автобуса это скорее не считают. Но это очень спорно. X, PUNCT, NUM в оценке постэггеров я учитывать не буду(!)

Кстати, дальше я делаю вот что: записываю токены и тэги в датасет, сохраняю его как табличку, скачиваю и аккуратно перепроверяю, чтобы улучшить свою разметку (очевидно, что изначально небольшие косяки были, надо было удостовериться, что они не попадут в окончательную версию разметки). Потом загружаю обновленный файлик с датасетом и вот уже с ним работаю дальше.

In [ ]:
# создание датасета
new_df = pd.DataFrame({'text': text_tokenized, 'POS': gold})

In [13]:
# создание файлика с датасетом
new_df.to_csv('text_gold.csv')

In [149]:
# обратно загрузка улучшенного датасета в ноут
df = pd.read_csv('text_gold_checked.csv', index_col=0)
df = df.dropna()
df

,text,POS
0.0,В,ADP
1.0,связи,NOUN
2.0,с,ADP
3.0,волной,NOUN
4.0,заселения,NOUN
...,...,...
332.0,благо,NOUN
333.0,нашего,DET
334.0,общего,ADJ
335.0,дома,NOUN


In [150]:
corpora = df.text
gold_POS = df.POS

pymorphy

In [151]:
morph = MorphAnalyzer()
pymorphy_tokens = [x for x in corpora]
pymorphy_tags = []
for token in pymorphy_tokens:
  parsed = morph.parse(token)[0]
  pymorphy_tags.append(parsed.tag.POS)
pymorphy_df = pd.DataFrame({'token': pymorphy_tokens, 'tag': pymorphy_tags})
pymorphy_df

,token,tag
0,В,PREP
1,связи,NOUN
2,с,PREP
3,волной,NOUN
4,заселения,NOUN
...,...,...
332,благо,NOUN
333,нашего,ADJF
334,общего,ADJF
335,дома,NOUN


natasha

In [152]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

natasha_tokens = []
natasha_tags = []

doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for tokenn in doc.tokens:
  natasha_tokens.append(tokenn.text)
  natasha_tags.append(tokenn.pos)

In [153]:
natasha_df = pd.DataFrame({'token': natasha_tokens, 'tag': natasha_tags})
natasha_df

,token,tag
0,В,ADP
1,связи,NOUN
2,с,ADP
3,волной,NOUN
4,заселения,NOUN
...,...,...
339,благо,NOUN
340,нашего,DET
341,общего,ADJ
342,дома,NOUN


spacy

In [154]:
spacy_tags = []
spacy_tokens = []
nlp = spacy.load('ru_core_news_sm')
doc = nlp(text)
for tok in doc:
  spacy_tokens.append(tok.text)
  spacy_tags.append(tok.pos_)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [155]:
spacy_df = pd.DataFrame({'token': spacy_tokens, 'tag': spacy_tags})
spacy_df

,token,tag
0,В,ADP
1,связи,NOUN
2,с,ADP
3,волной,NOUN
4,заселения,NOUN
...,...,...
342,благо,NOUN
343,нашего,DET
344,общего,ADJ
345,дома,NOUN


In [177]:
final_df = pd.DataFrame(columns=['token', 'gold', 'pymorphy', 'natasha', 'spacy'])
final_df

,token,gold,pymorphy,natasha,spacy


In [180]:
final_df['token'] = df.text
final_df['gold'] = df.POS
final_df['pymorphy'] = pymorphy_df.tag
final_df = final_df.reset_index(drop=True)
final_df

,token,gold,pymorphy,natasha,spacy
0,В,ADP,PREP,NaN,NaN
1,связи,NOUN,NOUN,NaN,NaN
2,с,ADP,PREP,NaN,NaN
3,волной,NOUN,NOUN,NaN,NaN
4,заселения,NOUN,NOUN,NaN,NaN
...,...,...,...,...,...
332,благо,NOUN,NOUN,NaN,NaN
333,нашего,DET,ADJF,NaN,NaN
334,общего,ADJ,ADJF,NaN,NaN
335,дома,NOUN,NOUN,NaN,NaN


Так как все модели, кроме pymorphy, потому что у него собственной нету, делали собственную токенизацию, для сравнения тегов нужен код, который будет дальше. Нужно посчитать условное отставание/опережение токена в списке модели по сравнению с токенами в списке из эталонной разметки (моей). Произойти это все может из-за слов с дефисами и двоеточиями. Надо учитывать эту разницу при сопоставлении, иначе все просто полетит. Код не очень красивый, но он работает. На всякий случай получившуюся таблицу тоже закидываю в файл, чтобы было.

In [181]:
for i in range(len(pymorphy_tokens)):
  if natasha_tokens[i] == pymorphy_tokens[i]:
    final_df.loc[i]['natasha'] = natasha_tags[i]
    final_df.loc[i]['spacy'] = spacy_tags[i]
  elif natasha_tokens[i] == pymorphy_tokens[i] or spacy_tokens[i] == pymorphy_tokens[i]:
    if natasha_tokens[i] == pymorphy_tokens[i]:
      final_df.loc[i]['natasha'] = natasha_tags[i]
    else:
      final_df.loc[i]['spacy'] = spacy_tags[i]
  else:
    for k in range(0, 5):
      if natasha_tokens[i + k] == pymorphy_tokens[i] and spacy_tokens[i + k] == pymorphy_tokens[i]:
        final_df.loc[i]['natasha'] = natasha_tags[i + k]
        final_df.loc[i]['spacy'] = spacy_tags[i + k]
      elif natasha_tokens[i + k] == pymorphy_tokens[i] or spacy_tokens[i + k] == pymorphy_tokens[i]:
        if natasha_tokens[i + k] == pymorphy_tokens[i]:
          final_df.loc[i]['natasha'] = natasha_tags[i + k]
        else:
          final_df.loc[i]['spacy'] = spacy_tags[i + k]
      elif (natasha_tokens[i + k] + natasha_tokens[i + k + 1] + natasha_tokens[i + k + 2] == pymorphy_tokens[i]) and (spacy_tokens[i + k] + spacy_tokens[i + k + 1] + spacy_tokens[i + k + 2] == pymorphy_tokens[i]):
        final_df.loc[i]['natasha'] = natasha_tags[i + k + 2]
        final_df.loc[i]['spacy'] = spacy_tags[i + k + 2]
      elif (natasha_tokens[i + k] + natasha_tokens[i + k + 1] + natasha_tokens[i + k + 2] == pymorphy_tokens[i]) or (spacy_tokens[i + k] + spacy_tokens[i + k + 1] + spacy_tokens[i + k + 2] == pymorphy_tokens[i]):
        if (natasha_tokens[i + k] + natasha_tokens[i + k + 1] + natasha_tokens[i + k + 2] == pymorphy_tokens[i]):
          final_df.loc[i]['natasha'] = natasha_tags[i + k + 2]
        else:
          final_df.loc[i]['spacy'] = spacy_tags[i + k + 2]


In [182]:
for i in range(1, len(pymorphy_tokens)):
  if natasha_tokens[len(natasha_tokens) - i] == pymorphy_tokens[len(pymorphy_tokens) - i] and spacy_tokens[len(spacy_tokens) - i] == pymorphy_tokens[len(pymorphy_tokens) - i]:
    final_df.loc[len(pymorphy_tokens) - i]['natasha'] = natasha_tags[len(natasha_tokens) - i]
    final_df.loc[len(pymorphy_tokens) - i]['spacy'] = spacy_tags[len(spacy_tokens) - i]
  elif natasha_tokens[len(natasha_tokens) - i] == pymorphy_tokens[len(pymorphy_tokens) - i] or spacy_tokens[len(spacy_tokens) - i] == pymorphy_tokens[len(pymorphy_tokens) - i]:
    if natasha_tokens[len(natasha_tokens) - i] == pymorphy_tokens[len(pymorphy_tokens) - i]:
      final_df.loc[len(pymorphy_tokens) - i]['natasha'] = natasha_tags[len(natasha_tokens) - i]
    else:
      final_df.loc[len(pymorphy_tokens) - i]['spacy'] = spacy_tags[len(spacy_tokens) - i]

In [183]:
# да, я смотрю вручную, где остались наны, и дописываю туда нужное
for i in range(273, 295):
  if spacy_tokens[i] == pymorphy_tokens[i]:
      final_df.loc[i]['spacy'] = spacy_tags[i]
  else:
    for k in range(0, 10):
      if spacy_tokens[i + k] == pymorphy_tokens[i]:
        final_df.loc[i]['spacy'] = spacy_tags[i + k]
      elif (spacy_tokens[i + k] + spacy_tokens[i + k + 1] + spacy_tokens[i + k + 2]) == pymorphy_tokens[i]:
        final_df.loc[i]['spacy'] = spacy_tags[i + k + 2]

In [185]:
for i in range(217, 229):
  if natasha_tokens[i] == pymorphy_tokens[i]:
      final_df.loc[i]['natasha'] = natasha_tags[i]
  else:
    for k in range(0, 10):
      if natasha_tokens[i + k] == pymorphy_tokens[i]:
        final_df.loc[i]['natasha'] = natasha_tags[i + k]
      elif (natasha_tokens[i + k] + natasha_tokens[i + k + 1] + natasha_tokens[i + k + 2]) == pymorphy_tokens[i]:
        final_df.loc[i]['natasha'] = natasha_tags[i + k + 2]

In [186]:
final_df

,token,gold,pymorphy,natasha,spacy
0,В,ADP,PREP,ADP,ADP
1,связи,NOUN,NOUN,NOUN,NOUN
2,с,ADP,PREP,ADP,ADP
3,волной,NOUN,NOUN,NOUN,NOUN
4,заселения,NOUN,NOUN,NOUN,NOUN
...,...,...,...,...,...
332,благо,NOUN,NOUN,NOUN,NOUN
333,нашего,DET,ADJF,DET,DET
334,общего,ADJ,ADJF,ADJ,ADJ
335,дома,NOUN,NOUN,NOUN,NOUN


In [187]:
final_df.to_csv('final_df.csv')

In [188]:
# final_df = pd.read_csv('final_df.csv', index_col=0)

In [189]:
# удаляем NUM, PUNCT, X из моей разметки, т. к. они не идут в подсчет accuracy, если они остались в разметке постеггеров - это вопрос к ним
final_df = final_df[~final_df.gold.isin(["NUM", "PUNCT", "X"])]
final_df

,token,gold,pymorphy,natasha,spacy
0,В,ADP,PREP,ADP,ADP
1,связи,NOUN,NOUN,NOUN,NOUN
2,с,ADP,PREP,ADP,ADP
3,волной,NOUN,NOUN,NOUN,NOUN
4,заселения,NOUN,NOUN,NOUN,NOUN
...,...,...,...,...,...
331,на,ADP,PREP,ADP,ADP
332,благо,NOUN,NOUN,NOUN,NOUN
333,нашего,DET,ADJF,DET,DET
334,общего,ADJ,ADJF,ADJ,ADJ


In [190]:
print(f'gold: {final_df.gold.unique()}')
print(f'pymorphy: {final_df.pymorphy.unique()}')
print(f'natasha: {final_df.natasha.unique()}')
print(f'spacy: {final_df.spacy.unique()}')

gold: ['ADP' 'NOUN' 'PRON' 'VERB' 'ADJ' 'CCONJ' 'DET' 'AUX' 'INTJ' 'ADV' 'PART'
 'SCONJ']
pymorphy: ['PREP' 'NOUN' 'NPRO' 'VERB' 'ADJF' 'PRCL' 'INFN' nan 'CONJ' 'ADVB' 'ADJS'
 'COMP']
natasha: ['ADP' 'NOUN' 'PRON' 'VERB' 'ADJ' 'ADV' 'DET' 'PROPN' 'CCONJ' 'AUX' 'PART'
 'SCONJ']
spacy: ['ADP' 'NOUN' 'PRON' 'VERB' 'ADJ' 'ADV' 'DET' 'PROPN' 'CCONJ' 'AUX' 'NUM'
 'PUNCT' 'PART' 'SCONJ']


pymorphy: ADP -> PREP, ADVB -> ADV, NPRO -> PRON, ADJF -> ADJ, ADJS -> ADJ, CONJ -> SCONJ/CCONJ, PRCL -> PART, INFN -> VERB, у него нет AUX, DET, поэтому для него эти случаи считаем как VERB и ADJ.

natasha & spacy: PROPN -> NOUN

In [192]:
final_df['comp_for_pymorph'] = final_df.gold.copy()
final_df.loc[final_df.comp_for_pymorph == 'DET', 'comp_for_pymorph'] = 'ADJ'
final_df.loc[final_df.comp_for_pymorph == 'SCONJ', 'comp_for_pymorph'] = 'CONJ'
final_df.loc[final_df.comp_for_pymorph == 'CCONJ', 'comp_for_pymorph'] = 'CONJ'
final_df.loc[final_df.comp_for_pymorph == 'AUX', 'comp_for_pymorph'] = 'VERB'
final_df.loc[final_df.pymorphy == 'ADVB', 'pymorphy'] = 'ADV'
final_df.loc[final_df.pymorphy == 'NPRO', 'pymorphy'] = 'PRON'
final_df.loc[final_df.pymorphy == 'ADJF', 'pymorphy'] = 'ADJ'
final_df.loc[final_df.pymorphy == 'ADJS', 'pymorphy'] = 'ADJ'
final_df.loc[final_df.pymorphy == 'PRCL', 'pymorphy'] = 'PART'
final_df.loc[final_df.pymorphy == 'PRED', 'pymorphy'] = 'ADV'
final_df.loc[final_df.pymorphy == 'INFN', 'pymorphy'] = 'VERB'
final_df.loc[final_df.pymorphy == 'PREP', 'pymorphy'] = 'ADP'
final_df.loc[final_df.natasha == 'PROPN', 'natasha'] = 'NOUN'
final_df.loc[final_df.spacy == 'PROPN', 'spacy'] = 'NOUN'
final_df = final_df.fillna('X')
final_df

,token,gold,pymorphy,natasha,spacy,comp_for_pymorph
0,В,ADP,ADP,ADP,ADP,ADP
1,связи,NOUN,NOUN,NOUN,NOUN,NOUN
2,с,ADP,ADP,ADP,ADP,ADP
3,волной,NOUN,NOUN,NOUN,NOUN,NOUN
4,заселения,NOUN,NOUN,NOUN,NOUN,NOUN
...,...,...,...,...,...,...
331,на,ADP,ADP,ADP,ADP,ADP
332,благо,NOUN,NOUN,NOUN,NOUN,NOUN
333,нашего,DET,ADJ,DET,DET,ADJ
334,общего,ADJ,ADJ,ADJ,ADJ,ADJ


In [195]:
from sklearn.metrics import accuracy_score

print(f'Точность разметки spacy - {(accuracy_score(final_df.gold, final_df.spacy))}')
print(f'Точность разметки natasha - {(accuracy_score(final_df.gold, final_df.natasha))}')
print(f'Точность разметки pymorphy - {(accuracy_score(final_df.comp_for_pymorph, final_df.pymorphy))}')

Точность разметки spacy - 0.9022556390977443
Точность разметки natasha - 0.9210526315789473
Точность разметки pymorphy - 0.9097744360902256


Лучше всех справилась natasha, поэтому берем ее.
Шаблоны, которые можно взять:

1. ADV + ADJ
2. ADJ + NOUN
3. не + VERB

Во-первых, это распространенные сочетания, во-вторых, пункт 3 - это все ситуации, когда без "не" мы получаем противоположное значение, что очень плохо для определения тональности, например. Для тональности будет важна и степень оценки прилагательным: "хорошо" и "очень хорошо" - разные вещи, второе вряд ли встретится в негативном отзыве. Сочетание прил + существительное тоже будет более показательно, потому что будет понятно, чему именно дается оценка.

In [202]:
def chunker(sequence, size):
    output = []
    for elem in range(0, len(sequence) - (size - 1)):
        output.append(sequence[elem: elem + size])
    return output

In [203]:
def bigrams(text):
    result = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for sht in chunker(doc.tokens, 2):
        if sht[0].pos == 'ADV' and sht[1].pos == 'ADJ':
            result.append(' '.join([sht[0].text, sht[1].text]))
        if sht[0].pos == 'ADJ' and sht[1].pos == 'NOUN':
            result.append(' '.join([sht[0].text, sht[1].text]))
    for sht in chunker(doc.tokens, 3):
        if sht[0].text == 'не' and sht[1].pos == 'VERB':
            result.append(' '.join([sht[0].text, sht[1].text]))
    return set(result)

In [204]:
bigrams('Я когда прихожу на ваш этаж, то вы предательски замолкаете, в итоге не могу понять, какая квартира. Очень страшный волк. Сейчас нахожусь в поиске личного помощника, для того, чтобы строить личный бренд и не только.')

{'Очень страшный',
 'личного помощника',
 'личный бренд',
 'не могу',
 'страшный волк'}

Все работает, ура! А дальше смотри файл HW_1_new в репо, где я прикрутила чанкер к самому хорошему варианту предсказаний результатов и, спойлер, стало лучше)